---
title: Graphs
order: 5
---

This post implements (and improves) code in chapter 14 from the textbook Introduction to Computation and Programming Using Python, With Application to Computational Modeling and Understanding Data, third edition, by John V. Guttag, 2020. 

In [1]:
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Tuple

In [2]:
@dataclass(frozen=True)
class Node:
    """Basic immutable graph node."""

    _name: str

    def get_name(self):
        """Returns node name."""
        return self._name

Designing the inheritance pattern is difficult to me here. The substitution principle: A code that works on a superclass must work on a subclass. But if a code works on bidirectional edges (perhaps exploiting symmetry) it may not work on unidirectional edges (where the symmetry doesn't exist). So, if anything, bidirectional edge must inherit from unidirectional edge. But that doesn't work either, because unidirectional edge logically must have the sources and destinations, but bidirectional edge cannot have them. Which node should be returned by a bidirectional edge's "source" property? There is no clear answer, so I conclude bidirectional edge cannot inherit from unidirectional. Together these two facts mean neither can inherit from the other.\

One solution would be to completely discard bidirectional edges and only work with directional ones - after all, we can build undirected graph with directed edges. But that would mean, while building an undirected graph, that we treat unidirectional edges as bidirectional, which is weird.

In [3]:
class BaseEdge(ABC):
    """Abstract base class for edge."""

    @abstractmethod
    def __init__(self, node1: Node, node2: Node, separator: str):
        self._node1 = node1
        self._node2 = node2
        self._separator = separator

    def __str__(self):
        sep = self._separator
        return f"{self._node1.get_name()} {sep} {self._node2.get_name()}"


class BaseWeightedEdge(BaseEdge):
    """Abstract base class for weighted edge."""

    @abstractmethod
    def __init__(self, node1: Node, node2: Node, weight: int | float):
        BaseEdge.__init__(self, node1, node2, "")
        self._weight = weight

    def get_weight(self):
        return self._weight


class Edge(BaseEdge):
    """Unweighted bidirectional edge."""

    def __init__(self, node1: Node, node2: Node):
        """Assumes node1 and node2 are Nodes."""
        BaseEdge.__init__(self, node1, node2, "<---->")

    def get_nodes(self):
        """Returns both nodes as a tuple."""
        return self._node1, self._node2

    def __eq__(self, other):
        """Two Edges are equal if they have the same nodes (order doesn't matter)."""
        if isinstance(other, Edge):
            return set(self.get_nodes()) == set(other.get_nodes())
        return False


class WeightedEdge(BaseWeightedEdge, Edge):
    """Weighted bidirectional edge."""

    def __init__(self, node1: Node, node2: Node, weight: int | float = 1):
        "Assumes node1 and node2 are Nodes."
        BaseWeightedEdge.__init__(self, node1, node2, weight)
        self._separator = f"<--({self.get_weight()})-->"

    def __eq__(self, other):
        if isinstance(self, WeightedEdge):
            weights_equal = self.get_weight() == other.get_weight()
            return Edge.__eq__(self, other) and weights_equal
        return False


class DirectedEdge(BaseEdge):
    """Unweighted unidirectional edge."""

    def __init__(self, src: Node, dest: Node):
        """Assumes src and dest are Nodes."""
        # maintaines the invariant that src=node1 and dest=node2
        BaseEdge.__init__(self, src, dest, "---->")

    def get_src(self):
        """Returns src."""
        return self._node1

    def get_dest(self):
        """Returns dest."""
        return self._node2

    def __eq__(self, other):
        """Two DirectedEdges are equal if sources equal and destinations equal."""
        if isinstance(self, DirectedEdge):
            sources_equal = self.get_src() == other.get_src()
            destinations_equal = self.get_dest() == other.get_dest()
            return sources_equal and destinations_equal
        return False


class WeightedDirectedEdge(BaseWeightedEdge, DirectedEdge):
    """Weighted unidirectional edge."""

    def __init__(self, src: Node, dest: Node, weight: int | float = 1):
        """Assumes src and dest Nodes, weight a number."""
        BaseWeightedEdge.__init__(self, src, dest, weight)
        self._separator = f"--({self.get_weight()})-->"

    def __eq__(self, other):
        """Two WeightedDirectedEdges are equal if also their weights are equal, in addition to sources and destinations."""
        if isinstance(self, WeightedDirectedEdge):
            weights_equal = self.get_weight() == other.get_weight()
            return DirectedEdge.__eq__(self, other) and weights_equal
        return False

In [4]:
# BaseEdge()
node1 = Node("1")
node2 = Node("2")

print(node1 == node2)

False


In [5]:
edgeA = Edge(node1, node2)
w_edgeA = WeightedEdge(node1, node2, 3.14)
dir_edgeA = DirectedEdge(node1, node2)
wdir_edgeA = WeightedDirectedEdge(node1, node2, 3.14)

In [6]:
edgeB = Edge(node2, node1)
w_edgeB = WeightedEdge(node2, node1, 3.14)
dir_edgeB = DirectedEdge(node2, node1)
wdir_edgeB = WeightedDirectedEdge(node2, node1, 3.14)

In [7]:
print(f"{edgeA == edgeB = }")
print(f"{w_edgeA == w_edgeB = }")
print(f"{dir_edgeA == dir_edgeB = }")
print(f"{wdir_edgeA == wdir_edgeB = }")

edgeA == edgeB = True
w_edgeA == w_edgeB = True
dir_edgeA == dir_edgeB = False
wdir_edgeA == wdir_edgeB = False
